In [1]:
import pickle as pkl
import random

from MENLI.MENLI import MENLI

scorer = MENLI(direction="avg", formula="e", src=False, nli_weight=0.3, combine_with="BERTScore-F", model="D")
res=[]
count=0
# refs and hyps in form of list of String
for cand, ref in tqdm(zip(p_data_mistral_cnn['highlights'],p_data_mistral_cnn['model_summaries'])):
    count+=1
    # if count>=20:
    #     break
    if len(cand)!=len(ref):
        continue
    if not cand or not ref:
        continue
    scores=scorer.score_all(srcs=[],refs=cand, hyps=ref) 

    res.append(sum(scores)/len(scores))
    
   
# return sum(res)/len(res)

ModuleNotFoundError: No module named 'MENLI'

In [9]:
with open('xsum_new_new.pkl', 'rb') as f:
    pkls_list=pkl.load(f)

In [10]:
cnn_list_capped=[]

for item in pkls_list:
    try:
        length=len(item)
        choose=random.randint(0,length-1)
        cnn_list_capped.append([item[0]])
    except:
        cnn_list_capped.append([])

In [11]:
cnn_list_capped[59]

['16 tie.']

In [12]:
with open('xsum_capped-random.pkl', 'wb') as f:
    pkl.dump(cnn_list_capped,f)

In [30]:
import pickle as pkl
import nltk
import os
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import evaluate
from rouge_score import rouge_scorer
from tqdm.auto import tqdm
from natsort import natsorted
import random
import pandas as pd
from datasets import load_dataset, DatasetDict, load_from_disk
import datasets

In [22]:
pkls=os.listdir('reddit')
pkls=natsorted(pkls)
pkls_list=[]
for pikl in pkls:
    with open('reddit/{}'.format(pikl),'rb') as f:
        file=pkl.load(f)
    if not file:
        file.append(' ')
    pkls_list.extend(file)

print(len(pkls_list))
print(pkls_list[0])

4214
1. As a second grader at a birthday celebration in primary school, the author accidentally knocked vanilla cupcakes onto the teacher, causing a scene and leading to accusations of ruining her shoes.


In [23]:
count=0
for idx,item in enumerate(pkls_list):
    if item == " ":
        print(idx)
        count+=1
print(count)

0


In [24]:
cnn_list=[]
for i in range(len(pkls_list)):
    temp=[]
    splits=pkls_list[i].split('\n')
    for split in splits:
        if split:
            if split[0].isdigit():
                try:
                    temp.append(split.split(".")[1].strip())
                except:
                    print(split)
    cnn_list.append(temp)

In [25]:
len(cnn_list)

4214

In [26]:
cnn_list[213]

["During lunch, a student's attempt to focus on copying notes led to a disruptive outburst that resulted in a 2-hour detention for causing a commotion in the lunchroom"]

In [27]:
with open('reddit_final.pkl', 'wb') as f:
    pkl.dump(cnn_list,f) 

In [108]:
# xsum_ds = datasets.load_dataset("EdinburghNLP/xsum")    
# xsum_ds = xsum_ds['test']

reddit_ds = datasets.load_dataset("reddit_tifu", "long")
train_testvalid = reddit_ds['train'].train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
reddit_ds = test_valid['test']

/home/haskari/miniconda3/envs/paraphrase/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for reddit_tifu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/reddit_tifu
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [109]:
# articles=xsum_ds['document']
# gold_summaries=xsum_ds['summary']

articles=reddit_ds['documents']
gold_summaries=reddit_ds['tldr']

In [110]:
# with open('xsum_final.pkl', 'rb') as f:
#     original_summaries=pkl.load(f)

with open('reddit_final.pkl', 'rb') as f:
    original_summaries=pkl.load(f)

In [111]:
original_summaries[43]

['The author experienced a situation where their dog became aggressive towards the mailman after they forgot to lock the dog away, leading to the dog escaping and the author getting locked out of their apartment for four hours']

In [116]:
# with open('../data_paraphrase/xsum.pkl', 'rb') as f:
#     paraphrased_summaries=pkl.load(f)

with open('../data_paraphrase/reddit.pkl', 'rb') as f:
    paraphrased_summaries=pkl.load(f)

In [117]:
paraphrased_summaries[0]

["The author, as a second grader, accidentally knocked vanilla cupcakes out of their teacher's hand, causing a scene in the cafeteria and leading to accusations of ruining the teacher's shoes"]

In [118]:
print(len(articles))
print(len(gold_summaries))
print(len(original_summaries))
print(len(paraphrased_summaries))

4214
4214
4214
4214


In [148]:
# articles=articles[-100:]
# gold_summaries=gold_summaries[-100:]
# original_summaries=original_summaries[-100:]
# paraphrased_summaries=paraphrased_summaries[-100:]

articles=articles[-90:]
gold_summaries=gold_summaries[-90:]
original_summaries=original_summaries[-90:]
paraphrased_summaries=paraphrased_summaries[-90:]

In [149]:
final_art=[]
final_gold=[]
final_orig=[]
final_para=[]

In [150]:
count=0
for art, gold, orig, para in zip(articles,gold_summaries,original_summaries,paraphrased_summaries):
    if art and gold and orig and para:
        # print(count)
        if count>=10:
            print('here')
            break
        final_art.append(art)
        final_gold.append(gold)
        final_orig.append(orig[0])
        final_para.append(para[0])
        count+=1
        
        

here


In [151]:
final_art

['okay so this happened a year ago. my buddy kade and i used to go to a gymnastics studio after school. we wrestled a lot. one day he had me on my back, knees up, and was going to stomp my knees. but his foot slipped, and it crushed my balls.',
 "very recently, i was travelling in wales with my girlfriend, her sister and her sister's 2 young daughters, aged 10 and 6. we were driving around on a very hot day, in a stuffy, cramped car. i was sitting in the back, next to the 6 year old in the middle, and the 10 year old on the other side, while my girlfriend was in the front passenger side, and her sister was driving. \n\nwe were about an hour into the drive and i was feeling a bit unwell. my stomach was churning and i was feeling light headed. also the music on in the car was a disney cd, on repeat. it was like one of dante's circles of hell. my gf's nieces were squealing with delight at the horrendous music being played on repeat, and i was losing the will to live. also i couldn't put t

In [152]:
final_gold

['wrestled with a second grader. got sterilized.',
 'i farted in a car with my niece, she got blamed for pooping herself.',
 "i'm an asshole.",
 'they made my fucking iced coffee out of straight espresso and i chugged it like an idiot',
 'got pics developed, tech saw my junk.',
 'i broke an expensive lab equipment (5000-10000 usd from what i found on the internet) at school.**',
 "trusted guy to drive us to secret jungle party, saving us some pocket change. guy didn't know how to drive on road, got flat tire, passport held hostage, had to cough up hundreds of dollars for a single flat tire.",
 'got drunk with members of a group that are on the u.s. list of terror organizations. may have helped fund terrorism inadvertently.',
 'went to chick-fil-a, straw wouldnt go in my drink. forced it in, poked a hole in the cup and got dr pepper all over my jeans, shoes and tray with my food.',
 'got high and put my wrong bank information into venmo, have been depositing my transactions into a compl

In [153]:
final_orig

["A year ago, while at a gymnastics studio with his friend Kade, the author experienced a painful accident when Kade's foot slipped and crushed his testicles during a wrestling match",
 "During a hot and uncomfortable car ride with his girlfriend's family, a man's discreet attempt to release a fart leads to a chaotic situation involving accusations of an accident and tears from a young girl",
 "During a high school band trip to Disney World, a student accidentally mocked a baby's whine, leading to an awkward and embarrassing encounter with the child's mother",
 "The article describes a person's accidental consumption of a highly caffeinated drink, mistaking it for iced coffee, leading to a distressing and mind-altering experience",
 'The author went on a vacation to LA with their girlfriend, got film developed from the trip, and discovered a photo of themselves urinating that they had no recollection of taking',
 "A graduate student in a lab accidentally used an old centrifuge incorrec

In [154]:
final_para

["During a wrestling match at a gymnastics studio, the author's friend accidentally injured them by crushing their genitals with his foot",
 'During a car trip with his girlfriend, her sister, and her young daughters, the author struggles with discomfort and an urge to fart, leading to an awkward incident where one of the girls has an accident and cries, likely due to a past experience of being reprimanded for dishonesty',
 "The author inadvertently mocked a baby's whine while in line for a ride at Disney World, causing embarrassment and receiving a horrified look from the child's mother",
 'The author accidentally consumed an intense iced coffee which turned out to be espresso in ice, causing an unpleasant and mind-altering experience',
 'The author went on vacation to Los Angeles with their girlfriend, got drunk and took photos with an ancient Olympus camera, but only discovered later that they had accidentally taken a picture of themselves urinating with their genitals visible when 

In [155]:
dic={"Article":final_art, "Gold":final_gold, "Original": final_orig, "Paraphrased": final_para}

In [156]:
df=pd.DataFrame.from_dict(dic)

In [157]:
# df.to_csv('../annotation/xsum_gpt.csv')

df.to_csv('../annotation/reddit_gpt.csv')